In [1]:
# Import libraries and load monthly stock price data

import numpy as np
import pandas as pd
from functools import reduce
rf= 0.05

files=['hdfc.csv','itc.csv','l&t.csv','m&m.csv','sunpha.csv','tcs.csv']
dfs=[]

for file in files:
    temp=pd.read_csv(file)
    temp.columns=['Date',file.replace('.csv','')]
    dfs.append(temp)

stocks = reduce(lambda left,right: pd.merge(left,right,on='Date'), dfs)
print(stocks.shape)
stocks.sort_index(ascending=False,inplace=True)
stocks.drop(labels=[60],axis=0,inplace=True)
stocks.reset_index(drop=True,inplace=True)
stocks.head()

(61, 7)


,Date,hdfc,itc,l&t,m&m,sunpha,tcs
0,01-07-2022,1406.150024,295.000000,1663.000000,1165.050049,857.250000,3113.800049
1,01-06-2022,1348.000000,273.500000,1558.250000,1093.150024,830.599976,3267.100098
2,01-05-2022,1388.949951,270.649994,1654.500000,1034.349976,860.599976,3364.350098
3,01-04-2022,1384.599976,259.549988,1694.349976,922.099976,928.650024,3546.699951
4,01-03-2022,1470.349976,250.649994,1767.650024,806.549988,914.750000,3739.949951


In [2]:
## Define all the functions required for Genetic Algorithm

def hist_return(months):
    ''' It calculates Stock returns for various months and returns a dataframe.
        Input: Months in the form of a list.
        Output: Historical returns in the form of a DataFrame. '''
    idx=[]
    df=pd.DataFrame()
    for mon in months:
        temp=(stocks.iloc[0,1:] - stocks.iloc[mon,1:])/(stocks.iloc[mon,1:])
        idx.append(str(mon)+'_mon_return')
        df=pd.concat([df, temp.to_frame().T], ignore_index=True)
    df.index=idx
    return df

##Mean portfolio return = Mean Return * Fractions of Total Capital (Chromosome). 
# Risk-free rate = 0.05 (assumption) 
# Standard deviation of portfolio return = (chromosome * Standard deviation)**2 + Covariance * Respective weights in chromosome.
def mean_portfolio_return(child):
    return np.sum(np.multiply(child,mean_hist_return))
def var_portfolio_return(child):
    part_1 = np.sum(np.multiply(child,sd_hist_return)**2)
    temp_lst=[]
    for i in range(6):
        for j in range(6):
            temp=cov_hist_return.iloc[i][j] * child[i] * child[j]
            temp_lst.append(temp)
    part_2=np.sum(temp_lst)
    return part_1+part_2

# Chromosome: Set of genes i.e. fractions of total capital assigned to each asset
def chromosome(n):
    ''' Generates set of random numbers whose sum is equal to 1
        Input: Number of stocks.
        Output: Array of random numbers'''
    ch = np.random.rand(n)
    return ch/sum(ch)

## Fitness function : The Sharpe ratio, S = (µ − r)/σ
## Maximize return, minimize risk
def fitness_fuction(child):
    ''' This will return the Sharpe ratio for a particular portfolio.
        Input: A child/chromosome (1D Array)
        Output: Sharpe Ratio value (Scalar)'''
    return (mean_portfolio_return(child)-rf)/np.sqrt(var_portfolio_return(child))

def Select_elite_population(population, frac=0.3):
    ''' Select elite population from the total population based on fitness function values.
        Input: Population and fraction of population to be considered as elite.
        Output: Elite population.'''
    population = sorted(population,key = lambda x: fitness_fuction(x),reverse=True)
    percentage_elite_idx = int(np.floor(len(population)* frac))
    return population[:percentage_elite_idx]

def mutation(parent):
    ''' Randomy choosen elements of a chromosome are swapped
        Input: Parent
        Output: Offspring (1D Array)'''
    child=parent.copy()
    n=np.random.choice(range(6),2)
    while (n[0]==n[1]):
        n=np.random.choice(range(6),2)
    child[n[0]],child[n[1]]=child[n[1]],child[n[0]]
    return child

def Heuristic_crossover(parent1,parent2):
    ''' The oﬀsprings are created according to the equation:
            Off_spring A = Best Parent  + β ∗ ( Best Parent − Worst Parent)
            Off_spring B = Worst Parent - β ∗ ( Best Parent − Worst Parent)
                Where β is a random number between 0 and 1.
        Input: 2 Parents
        Output: 2 Children (1d Array)'''
    ff1=fitness_fuction(parent1)
    ff2=fitness_fuction(parent2)
    diff=parent1 - parent2
    beta=np.random.rand()
    if ff1>ff2:
        child1=parent1 + beta * diff
        child2=parent2 - beta * diff
    else:
        child2=parent1 + beta * diff
        child1=parent2 - beta * diff
    return child1,child2

def Arithmetic_crossover(parent1,parent2):
    ''' The oﬀsprings are created according to the equation:
            Off spring A = α ∗ Parent1 + (1 −α) ∗ Parent2
            Off spring B = (1 −α) ∗ Parent1 + α ∗ Parent2
            
                Where α is a random number between 0 and 1.
        Input: 2 Parents
        Output: 2 Children (1d Array)'''
    alpha = np.random.rand()
    child1 = alpha * parent1 + (1-alpha) * parent2
    child2 = (1-alpha) * parent1 + alpha * parent2
    return child1,child2

def next_generation(pop_size,elite,crossover=Arithmetic_crossover):
    ''' Generates new population from elite population with mutation probability as 0.4 and crossover as 0.6. 
        Over the final stages, mutation probability is decreased to 0.1.
        Input: Population Size and elite population.
        Output: Next generation population (2D Array).'''
    new_population=[]
    elite_range=range(len(elite))
    #print(elite_range)
    while len(new_population) < pop_size:
        if len(new_population) > 2*pop_size/3: # In the final stages mutation frequency is decreased.
            mutate_or_crossover = np.random.choice([0, 1], p=[0.9, 0.1])
        else:
            mutate_or_crossover = np.random.choice([0, 1], p=[0.4, 0.6])
        #print(mutate_or_crossover)
        if mutate_or_crossover:
            indx=np.random.choice(elite_range)
            new_population.append(mutation(elite[indx]))
            #print('Mutation!')
        else:
            p1_idx,p2_idx=np.random.choice(elite_range,2)
            c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            chk=0
            for gene in range(6):
                if c1[gene]<0:
                    chk+=1
                else:
                    chk+=0
            if chk>0:
                p1_idx,p2_idx=np.random.choice(elite_range,2)
                c1,c2=crossover(elite[p1_idx],elite[p2_idx])
            
            new_population.extend([c1,c2])
            #print('Crossover!')    
    return new_population


hist_stock_returns=hist_return([3,6,12,24,36])
#print(hist_stock_returns.info())
cols=hist_stock_returns.columns
hist_stock_returns[cols] = hist_stock_returns[cols].apply(pd.to_numeric, errors='coerce')
#print(hist_stock_returns.info())

mean_hist_return=hist_stock_returns.mean()
sd_hist_return=hist_stock_returns.std()
cov_hist_return=hist_stock_returns.cov()
for i in range(6):
    cov_hist_return.iloc[i][i]=0


    
## Determine the Optimum portfolio using Arithmetic Crossover  

n=6 # Number of stocks = 6
pop_size=100 # initial population = 100

# Initial population
population = np.array([chromosome(n) for _ in range(pop_size)])

# Get initial elite population
elite = Select_elite_population(population)

iteration=0 
Expected_returns=0
Expected_risk=1

while (Expected_returns < 0.30 or Expected_risk > 0.04) and iteration <= 50:
    print('Iteration:',iteration)
    population = next_generation(100,elite)
    elite = Select_elite_population(population)
    Expected_returns=mean_portfolio_return(elite[0])
    Expected_risk=var_portfolio_return(elite[0])
    print('Expected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))
    iteration+=1

print('\n\n=====================================================\nPortfolio of stocks after all the iterations:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(6))]

print('\nExpected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))



Iteration: 0
Expected returns of 0.34438109994629595 with risk of 0.043215036713671204

Iteration: 1
Expected returns of 0.34432103829671934 with risk of 0.043320434812615105

Iteration: 2
Expected returns of 0.36088473816231686 with risk of 0.046228804528155

Iteration: 3
Expected returns of 0.34401205173418276 with risk of 0.04202839656187994

Iteration: 4
Expected returns of 0.3465143799957026 with risk of 0.042970623967329286

Iteration: 5
Expected returns of 0.35058251774623406 with risk of 0.04455449104965099

Iteration: 6
Expected returns of 0.3486595638039946 with risk of 0.043736669763345934

Iteration: 7
Expected returns of 0.34879801687134027 with risk of 0.04378255707378802

Iteration: 8
Expected returns of 0.3661275748734824 with risk of 0.048440096089486705

Iteration: 9
Expected returns of 0.3661802115072936 with risk of 0.04849859741205524

Iteration: 10
Expected returns of 0.3630720384889552 with risk of 0.047593366227955375

Iteration: 11
Expected returns of 0.3630590

In [3]:
## Determine the Optimum portfolio using Heuristic Crossover

n=6 # Number of stocks = 6
pop_size=100 # initial population = 100

# Initial population
population = np.array([chromosome(n) for _ in range(pop_size)])

# Get initial elite population
elite = Select_elite_population(population)

iteration=0 
Expected_returns=0
Expected_risk=1

while (Expected_returns < 0.30 or Expected_risk > 0.02):#and iteration <= 300
    print('Iteration:',iteration)
    population = next_generation(100,elite,Heuristic_crossover)
    elite = Select_elite_population(population)
    Expected_returns=mean_portfolio_return(elite[0])
    Expected_risk=var_portfolio_return(elite[0])
    print('Expected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))
    iteration+=1
print('\n\n=====================================================\nPortfolio of stocks after all the iterations:\n')
[print(hist_stock_returns.columns[i],':',elite[0][i]) for i in list(range(6))]

print('\nExpected returns of {} with risk of {}\n'.format(Expected_returns,Expected_risk))

Iteration: 0
Expected returns of 0.3387039533751445 with risk of 0.04718559745530146

Iteration: 1
Expected returns of 0.33031564806085106 with risk of 0.030556972265569624

Iteration: 2
Expected returns of 0.492728970725487 with risk of 0.07111164951392397

Iteration: 3
Expected returns of 0.842168789302923 with risk of 0.058158213577873596

Iteration: 4
Expected returns of 0.8543543338703274 with risk of 0.05909281101495806

Iteration: 5
Expected returns of 1.3719450683438221 with risk of 0.13749710172154828

Iteration: 6
Expected returns of 0.9682732610981358 with risk of 0.050067608165278

Iteration: 7
Expected returns of 0.9682732610981358 with risk of 0.050067608165278

Iteration: 8
Expected returns of 0.746031697442627 with risk of 0.027005807000034354

Iteration: 9
Expected returns of 0.8222284601856725 with risk of 0.019722043662927735



Portfolio of stocks after all the iterations:

hdfc : 0.9624242743998885
itc : 0.22754872038280088
l&t : -0.7250573761101828
m&m : 2.0798771